In [ ]:
import numpy as np
from astropy.table import Table, join, MaskedColumn, vstack, Column
import matplotlib.pyplot as plt
import emcee
from numpy import exp
from scipy import integrate
from scipy.integrate import quad
import math

In [ ]:
Cat=Table.read('Cat_to_Run_CMF_On.fits')

In [ ]:
#Definging necesary funcitons 

def pobs(M, mlim):
    k=6.02
    
    y=(1.+ exp(-k*(M-mlim)))**(-1)
    
    return y

def lnobs_like(M, mlim):
    k=6.02
    
    return -np.log(1.+ exp(-k*(M-mlim)))
        
def Shecter_Z(M, mlim, alpha, M_c):
    x = M/M_c
    k=6.02
    pobs= 1./(1.+ exp((-k)*(np.log10(M)-mlim)))
    
    return (x**alpha) * exp(-x) * pobs
        
    
def lnlike(theta, M, mlim):
    alpha, M_c = theta
    lin_M_c= 10.**M_c
    
    lin_M= 10**M
    x= lin_M/lin_M_c
    
    
    ln_pobs=lnobs_like(M, np.log10(mlim))
    
    norm= np.zeros(len(M))
    err=np.zeros(len(M))
    for i in range(len(M)):
        
        norm[i], err[i] = quad(Shecter_Z, mlim[i], 1.e7, args=(np.log10(mlim[i]), alpha, lin_M_c))
    
    
    lnlike = np.sum((-x) + alpha*np.log(x) + ln_pobs - np.log(norm))    
    return lnlike
    
def lnprior(theta):
    alpha, M_c = theta
    if -3 <= alpha <= -1 and 3 <= M_c <= 8:
        return 0.0
    return -np.inf

def lnprob(theta, M, mlim):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, M, mlim)

In [ ]:
#Doing the completeness cuts before putting the clusters into the function

#Ob_Com needs to be in log space 

def c(NMS):
    m=0.7727
    b=0.6674
    
    y= (m*NMS)+b
    
    if NMS < 2.53:
        return 2.7739
    if 2.53 <= NMS <= 3.49: 
        return y
    if NMS > 3.49:
        return 3.4694

def M_lim(Tau, NMS):
    
    #fit from completeness limit 
    a=0.0303
    b=1.9899
    
    c_=c(NMS)
    
    Tau_min=7.09

    y= a*np.exp(b*(Tau-Tau_min))+c_
    return y
    

In [ ]:
#Only using the sample in my age range

masses=np.array(Cat['Bestfit_Mass'])
ages=np.array(Cat'Bestfit_Age'])
nmses=np.array(np.log10(Cat['NMS']))

m_lims=np.zeros((len(ages)))

for i in range(len(ages)):
    m_lims[i]=M_lim(ages[i], nmses[i])
    
use_masses=np.array(masses[np.where(masses > m_lims)])
use_ages=ages[np.where(masses > m_lims)]
use_mlims=np.array(10**m_lims[np.where(masses > m_lims)])


In [ ]:
#Run MCMC

starting_point=np.array([-2., 4.2])

ndim, nwalkers = 2, 500
nsteps= 600
burnin=100
pos = starting_point + 1e-2*np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(use_,ages, use_mlims))
sampler.run_mcmc(pos, nsteps)

sampler.chain
samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))
fig = corner.corner(samples, labels=["$\alpha$", "$log(M_c)$"], label_kwargs={"fontsize": 16},
                                     quantiles=[0.16, 0.5, 0.84], show_titles=True, title_kwargs={"fontsize": 16})

corner.corner.savefig('Corner_plot.png')

alphas=[i[0] for i in samples]
Mcs= [i[1] for i in samples]

Samples_Table=Table([alphas, Mcs], names=('Alphas', 'MCs'))

Sample_Table.write('Samples_Table.fits')